In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_484612/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_484612/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_484612/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_484612/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
0,\1INCHUSDT\plots\2022-02-03T232328\backtest_re...,5.3,1INCHUSDT,0.909573,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,\1INCHUSDT\plots\2022-02-03T235304\backtest_re...,5.3,1INCHUSDT,0.114573,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,\1INCHUSDT\plots\2022-02-03T235442\backtest_re...,5.3,1INCHUSDT,0.838780,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,\1INCHUSDT\plots\2022-02-04T143331\backtest_re...,5.3,1INCHUSDT,0.952475,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,\1INCHUSDT\plots\2022-02-04T144601\backtest_re...,5.3,1INCHUSDT,0.242079,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,\ZECUSDT\plots\2022-02-22T131907\backtest_resu...,5.4,ZECUSDT,0.966600,True,True,0.0600,0.0179,100.0,100.0,0.9792,12.530,3.573
777,\ZECUSDT\plots\2022-02-22T134640\backtest_resu...,5.4,ZECUSDT,0.966600,True,True,0.0600,0.0179,100.0,100.0,0.9792,12.530,3.573
778,\ZECUSDT\plots\2022-02-25T222007\backtest_resu...,5.5,ZECUSDT,0.966000,True,True,0.0599,0.0173,100.0,100.0,0.9792,12.520,3.435
779,\ZECUSDT\plots\2022-02-25T232208\backtest_resu...,5.5,ZECUSDT,0.966000,True,True,0.0599,0.0173,100.0,100.0,0.9792,12.520,3.435


Now filter the dataframe

In [6]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["ONTUSDT","OGNUSDT","BLZUSDT","DASHUSDT"]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              & (big_data['adgt_%'] > 0.09)
              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)
              & (big_data['ebrs'] > .85)
              & (big_data['ebrl'] > .85)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%,adgt_%
147,\BLZUSDT\plots\2022-03-05T133505\backtest_resu...,5.5,BLZUSDT,0.9418,True,True,0.0869,0.1000,100.0,100.0,0.8790,20.220,23.13,0.1869
564,\OGNUSDT\plots\2022-03-05T080703\backtest_resu...,5.5,OGNUSDT,0.9304,True,True,0.0416,0.0881,100.0,100.0,0.9051,9.115,22.47,0.1297
565,\OGNUSDT\plots\2022-03-05T132424\backtest_resu...,5.5,OGNUSDT,0.9129,True,True,0.0483,0.0958,100.0,100.0,0.9176,10.800,24.97,0.1441
604,\ONTUSDT\plots\2022-03-05T075831\backtest_resu...,5.5,ONTUSDT,0.9414,True,True,0.0515,0.0674,100.0,100.0,0.9434,11.650,16.74,0.1189
605,\ONTUSDT\plots\2022-03-05T132111\backtest_resu...,5.5,ONTUSDT,0.9414,True,True,0.0515,0.0674,100.0,100.0,0.9434,11.650,16.74,0.1189


In [7]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_484612/4194297404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
147,\BLZUSDT\plots\2022-03-05T133505\backtest_resu...,BLZUSDT,0.1869,0.0869,0.1000,100.0,100.0,0.9418,0.8790
564,\OGNUSDT\plots\2022-03-05T080703\backtest_resu...,OGNUSDT,0.1297,0.0416,0.0881,100.0,100.0,0.9304,0.9051
565,\OGNUSDT\plots\2022-03-05T132424\backtest_resu...,OGNUSDT,0.1441,0.0483,0.0958,100.0,100.0,0.9129,0.9176
604,\ONTUSDT\plots\2022-03-05T075831\backtest_resu...,ONTUSDT,0.1189,0.0515,0.0674,100.0,100.0,0.9414,0.9434
